In [ ]:
try:
    import BACKEND_LIBRARY_IN  # noqa: F401
except ImportError:
    import ngsolve
else:
    try:
        import ngsolve
    except ImportError:
        !wget "https://PROJECT_NAME_IN.github.io/releases/ngsolve-install.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
        import ngsolve  # noqa: F401

In [ ]:
from mpi4py.MPI import COMM_WORLD
from ngsolve import *
from ngsPETSc import KrylovSolver

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.5).Distribute(COMM_WORLD))

In [ ]:
fes = H1(mesh, order=1)

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += u * v * dx
a.Assemble()

In [ ]:
gff = GridFunction(fes)
for i in range(fes.ndof):
    gff.vec[i] = i + 1

In [ ]:
F = LinearForm(fes)
F += gff * v * dx

In [ ]:
solver = KrylovSolver(a, fes, solverParameters={"ksp_type": "preonly", "pc_type": "lu"})
gfu = solver.solve(F)

In [ ]:
import numpy as np

In [ ]:
for i in range(fes.ndof):
    assert np.isclose(gfu.vec[i], i + 1)